In [66]:
import requests
from   bs4 import BeautifulSoup
import pandas

report_ary = []

with requests.Session() as s:
    s.headers.update({
        'user-agent': 'Mozilla/5.0'})

    r = s.get('http://www.jct.org.tw/tjcha_cert/ema.aspx')

    with open('page-1.html', 'bw') as log:
        log.write(r.content)

    soup = BeautifulSoup(r.content, 'html.parser')

    viewstate  = soup.find('input', {'id': '__VIEWSTATE'         })['value']
    generator  = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})['value']
    validation = soup.find('input', {'id': '__EVENTVALIDATION'   })['value']
    
    dfs=pandas.read_html(soup.select('#GridView1')[0].prettify())
    df=dfs[0]
    df.columns = df.iloc[0,]
    df = df.drop(0)
    df =df[:-2]
    df=df[(['序號', '醫療機構代碼', '機構名稱', '縣市別', '評定等級', '效期', '機構電話', '機構地址'])]
    report_ary.append(df)

    for page in range(2, 11):
        print(page)
        r = s.post(
            'http://www.jct.org.tw/tjcha_cert/ema.aspx',
            data={ 
                '__EVENTTARGET':        'GridView1',
                '__EVENTARGUMENT':      'Page${}'.format(page),
                '__VIEWSTATE':          viewstate,
                '__VIEWSTATEGENERATOR': generator,
                '__EVENTVALIDATION':    validation,
                'DropDownList1':'%',
                'DropDownList2':'%'
            }
        )

        with open('page-{}.html'.format(page), 'bw') as log:
            log.write(r.content)

        soup = BeautifulSoup(r.content, 'html.parser')

        viewstate  = soup.find('input', {'id': '__VIEWSTATE'      })['value']
        generator  = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})['value']
        validation = soup.find('input', {'id': '__EVENTVALIDATION'})['value']
        
        dfs=pandas.read_html(soup.select('#GridView1')[0].prettify())
        df=dfs[0]
        df.columns = df.iloc[0,]
        df = df.drop(0)
        df =df[:-2]
        df=df[(['序號', '醫療機構代碼', '機構名稱', '縣市別', '評定等級', '效期', '機構電話', '機構地址'])]
        report_ary.append(df)
reportdf = pandas.concat(report_ary)

        

2
3
4
5
6
7
8
9
10


In [68]:
reportdf.to_excel('erhosp.xlsx')